<a href="https://colab.research.google.com/github/Mohan1github/fine-tuned/blob/main/Fine_tuning_llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trl


import os
import torch
from datasets import load_dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer




In [2]:
!pip install -q datasets
!huggingface-cli login
!pip install -U datasets


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [3]:
import re

from datasets import load_dataset

dataset = load_dataset('timdettmers/openassistant-guanaco')

dataset = dataset['train'].shuffle(seed=42).select(range(1000))

def transform_function(example):
  conversation_text = example["text"]
  segments = conversation_text.split("###")

  reformatted_segments = []

  for i in range(1,len(segments)-1,2):
    human_text = segments[i].strip().replace("Human:",'').strip()

    if i+1 < len(segments):
      assistant_text = segments[i+1].strip().replace("Assistant:",'').strip()
      reformatted_segments.append(f"<s>[INST] {human_text} [/INST]  {assistant_text}")

    else:
       reformatted_segments.append(f"<s>[INST] {human_text} [/INST]")

  return {"text": reformatted_segments}


transformed_dataset = dataset.map(transform_function)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [4]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"

new_model = "Llama-2-7b-chat-finetune"


lora_r = 64

lora_alpha = 16

lora_dropout = 0.1

use_4bits = True

bnb_4bit_compute_dtype = "float16"
bnb_4bits_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"

num_train_epochs = 1

fp16 = False

bf16 = False

per_device_train_batch_size = 4

per_device_eval_batch_size = 4

gradient_accumulation_steps = 1

gradient_checkpointing = True
max_grad_norm = 0.34

learning_decay = 2e-4

weight_decay = 0.001

optim = "paged_adamw_32bit"

lr_scheduler_type = "cosine"

max_steps = -1

warmup_ratio = 0.03

group_by_length = True

save_steps = 0

logging_steps = 25





max_seq_lenght = None

picking = False

device_map = {"":0}









In [5]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00


In [19]:
from ast import arguments
import torch
import os # Import os to set environment variable

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from datasets import load_dataset # Import load_dataset here if it's not already imported in this cell
from trl import SFTTrainer # Import SFTTrainer

# Ensure trl is updated to a version that supports dataset_text_field
# !pip install -U trl # Moved this install to an earlier cell

# Set the environment variable to potentially help with fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


# Use the already loaded and transformed dataset
# dataset = load_dataset(dataset_name, split="train")

from ast import arguments
import torch
import os # Import os to set environment variable

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from datasets import load_dataset # Import load_dataset here if it's not already imported in this cell
from trl import SFTTrainer # Import SFTTrainer

# Ensure trl is updated to a version that supports dataset_text_field
# !pip install -U trl # Moved this install to an earlier cell

# Set the environment variable to potentially help with fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


# Use the already loaded and transformed dataset
# dataset = load_dataset(dataset_name, split="train")

# Check if CUDA is available and set the device accordingly
# If CUDA is not available, the code will run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Only attempt to get device capability if CUDA is available
if torch.cuda.is_available():
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bits,
        bnb_4bit_quant_type=bnb_4bits_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    if compute_dtype == torch.float16 and use_4bits:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
else:
    # If no CUDA device, set bnb_config to None or a CPU compatible config
    # For CPU training, 4-bit quantization might not be applicable or beneficial
    bnb_config = None
    # Ensure use_4bits is False for CPU
    use_4bits = False


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_cache=False,
    # Apply quantization only if CUDA is available and use_4bits is True
    quantization_config=bnb_config if torch.cuda.is_available() and use_4bits else None,
    # Remove device_map during loading and move to device explicitly after
    # device_map=device_map if torch.cuda.is_available() and use_4bits else None # Remove this line
)

# Move model to the selected device (CPU or GPU) AFTER loading
model.to(device)


model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code = True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_size = "right"


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Adjust TrainingArguments for CPU if necessary.
# For example, gradient_checkpointing can sometimes cause issues on CPU, but usually works.
# Batch sizes might need to be adjusted depending on CPU memory.
# Reduce batch size and disable gradient checkpointing
# Ensure this batch size is used in the TrainingArguments below
per_device_train_batch_size_adjusted = 1 # Reduced batch size
gradient_checkpointing = False # Disable gradient checkpointing

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    # Use the adjusted batch size
    per_device_train_batch_size=per_device_train_batch_size_adjusted,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_decay,
    weight_decay=weight_decay,
    # Set fp16 and bf16 to False for CPU training
    fp16 = False,
    bf16 = False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=gradient_checkpointing, # Apply the changed value
    # Add or modify device if needed, though Trainer often handles this.
    # device=device # Can explicitly set device here
)

# Use the transformed dataset instead of the original one
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset, # Use the transformed dataset
    peft_config=peft_config,
    dataset_text_field="text",
    # Change max_seq_length to dataset_max_seq_length or seq_length
    # Let's try dataset_max_seq_length first, as it's more likely for dataset processing
    # If this still fails, try seq_length

    dataset_max_seq_length=max_seq_lenght, # Changed argument name
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

trainer.train()
# Only attempt to get device capability if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Only attempt to get device capability if CUDA is available
if torch.cuda.is_available():
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bits,
        bnb_4bit_quant_type=bnb_4bits_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    if compute_dtype == torch.float16 and use_4bits:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
else:
    # If no CUDA device, set bnb_config to None or a CPU compatible config
    # For CPU training, 4-bit quantization might not be applicable or beneficial
    bnb_config = None
    # Ensure use_4bits is False for CPU
    use_4bits = False

# Load the model initially to CPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_cache=False,
    # Apply quantization only if CUDA is available and use_4bits is True
    # Pass the bnb_config here for 4-bit loading
    quantization_config=bnb_config if torch.cuda.is_available() and use_4bits else None,
    # Explicitly load to CPU first
    device_map="cpu",
)

# Move model to the selected device (CPU or GPU) AFTER loading and quantization
# If CUDA is available and use_4bits is True, the model is already quantized.
# Otherwise, it's loaded to CPU.
if torch.cuda.is_available():
    model.to(device)


model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code = True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_size = "right"


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Adjust TrainingArguments for CPU if necessary.
# For example, gradient_checkpointing can sometimes cause issues on CPU, but usually works.
# Batch sizes might need to be adjusted depending on CPU memory.
# Reduce batch size and disable gradient checkpointing
# Ensure this batch size is used in the TrainingArguments below
per_device_train_batch_size_adjusted = 1 # Reduced batch size
gradient_checkpointing = False # Disable gradient checkpointing

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    # Use the adjusted batch size
    per_device_train_batch_size=per_device_train_batch_size_adjusted,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_decay,
    weight_decay=weight_decay,
    # Set fp16 and bf16 to False for CPU training
    fp16 = False,
    bf16 = False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=gradient_checkpointing, # Apply the changed value
    # Add or modify device if needed, though Trainer often handles this.
    # device=device # Can explicitly set device here
)

# Use the transformed dataset instead of the original one
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset, # Use the transformed dataset
    peft_config=peft_config,
    dataset_text_field="text",
    # Change max_seq_length to dataset_max_seq_length or seq_length
    # Let's try dataset_max_seq_length first, as it's more likely for dataset processing
    # If this still fails, try seq_length

    dataset_max_seq_length=max_seq_lenght, # Changed argument name
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

trainer.train()


Using device: cuda


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-hf.
403 Client Error. (Request ID: Root=1-6827d8a4-2d07d07f6e9de9f4387c1c57;3e9e8312-7e6b-42a7-97b1-2bc35057adcc)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-hf to ask for access.